<a href="https://colab.research.google.com/github/niikun/Agent_for_practical_use/blob/main/conversationAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatInterfaceを使えばGradioでチャット画面が生成できる

In [2]:
!pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 kB 20.2 MB/s eta 0:00:00


In [3]:
import os
from google.colab import userdata
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["ANTHROPIC_API_KEY"] = userdata.get('ANTHROPIC_API_KEY')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()
MODEL = "gpt-4o-mini"

In [5]:
system_message = "あなたは親切なアシスタントです"

In [6]:
messages = [
    {"role":"system","content":system_message},
    {"role":"user", "content":"こんにちは"}
]

In [7]:
chat_openai = openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

In [8]:
chat_openai

ChatCompletion(id='chatcmpl-CFcBDedDzpBSJ9r4LqCKA1JOOgghG', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='こんにちは！今日はどんなお手伝いができるでしょうか？', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1757837719, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=15, prompt_tokens=22, total_tokens=37, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [12]:
chat_openai.choices[0].message.content

'こんにちは！今日はどんなお手伝いができるでしょうか？'

## Gradioがhistoryを作成してくれる

history がなくても Gradio が内部で会話履歴を管理して、必要な形式で関数に渡してくれているからです。

流れを整理すると：

Gradio の ChatInterface や gr.Chatbot コンポーネントは、ユーザーが入力したテキストや AI の返答を自動的に履歴（history）として保持しています。

例えば：
```
[
    ["こんにちは", "こんにちは！何をお手伝いしましょうか？"],
    ["天気教えて", "今日は晴れです☀️"]
]
```

こんな感じで「ユーザー発話」と「アシスタント応答」をペアで保存しています。

その history を、あなたの chat(message, history) 関数に毎回渡してくれる仕組みになっています。

最初の呼び出しのときは history=[]（空リスト）。

だから history がなくても（空でも）、Gradio はちゃんと動きます。

あなたのコードでは、その history を OpenAI の形式に変換して messages に追加しています。
```
messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
```

最初は history=[] だから、messages には system_message と最新の user だけ。

2回目以降は、Gradio が history を渡してくれるので、会話の流れを再現できる。

### まとめ

Gradio が自動で history を管理 → 関数に渡す ので、空のときでも表示できる。

つまり「自分で履歴を保存しなくても Gradio が勝手にやってくれる」仕組み。

In [20]:
def chat(message, history):

    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)
    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [23]:
messages

[{'role': 'system', 'content': 'あなたは親切なアシスタントです'},
 {'role': 'user', 'content': 'こんにちは'}]

In [21]:
gr.ChatInterface(fn=chat, type="messages").launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aa02e26c35c24052a9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# システムメッセージの更新

In [15]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [16]:
gr.ChatInterface(fn=chat, type="messages").launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://30bdd7596713a5778d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Fixed a bug in this function brilliantly identified by student Gabor M.!
# I've also improved the structure of this function

def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [17]:
gr.ChatInterface(fn=chat, type="messages").launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://554883c232e11c49ab.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
